In [ ]:
import random
import os
from groq import Groq
import minsearch
from dotenv import load_dotenv
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import requests
import tiktoken
import chat_assistant

In [ ]:
known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

### Q1 - define function description

In [ ]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather", # name of the function to invoke
    "description": "Search for the current weather in a city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": { # input 
                "type": "string",
                "description": "The name of the city to get the weather for, provided as a string. For example, 'Berlin'"
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

In [ ]:
tools = chat_assistant.Tools()
tools.add_tool(get_weather, get_weather_tool)
tools.get_tools()

developer_prompt = """
You are a helpful assistant that can answer questions about the weather in different cities.
You can use the `get_weather` function to find the current weather in a city.
You will be provided with a city name, and you should return the current weather in that city.
You should only use the `get_weather` function when necessary, and you should not make up answers.
You should always return the weather in degrees Celsius.
You should not use any other tools or functions, and you should not make up answers.
If you do not know the answer, you should say that you do not know adn search for thw eatehr on the web.
""".strip()

chat_interface = chat_assistant.ChatInterface()

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client,
    model="llama-3.3-70b-versatile"
)

In [ ]:
chat.run()

### Q2 - add new tool to RAG

In [ ]:

set_weather_description = {
    "type": "function",
    "name": "set_weather", # name of the function to invoke
    "description": "Add the current weather for a city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city to set the weather for, provided as a string. For example, 'Berlin'",
            },
            "temp": {
                "type": "number",
                "description": "The temperature in degrees Celsius to set for the city, provided as a float. For example, 20.5",
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}

tools.add_tool(set_weather, set_weather_description)
tools.get_tools()

In [ ]:
chat.run()

### Q3 - Install FastMCP

In [ ]:
!pip install fastmcp

In [ ]:
import fastmcp
fastmcp.__version__

### Q4 - simple fastmcp server


In [ ]:
# answer is STDIO, can't run in a notebook

### Q5 - protocol

In [ ]:
# the input should be 
# {"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "get_weather", "arguments": {"city": "Berlin"}}}

# this is what we get back from the server
# {"jsonrpc":"2.0","id":3,"result":{"content":[{"type":"text","text":"20.0"}],"structuredContent":{"result":20.0},"isError":false}}

### Q6 - Client

In [ ]:
import weather_server
from fastmcp import Client
import asyncio

async def main():
    async with Client(weather_server.mcp) as mcp_client:
        tools = await mcp_client.list_tools()
        print("Available tools:")
        print(tools)
        return tools

import nest_asyncio
nest_asyncio.apply() 

result = asyncio.run(main())

Available tools:
[Tool(name='get_weather', title=None, description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None), Tool(name='set_weather', title=None, description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'tem

### Optional

In [4]:
import mcp_client

our_mcp_client = mcp_client.MCPClient(["python", "weather_server.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

Started server with command: python weather_server.py
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Demo 🚀', 'version': '1.11.0'}}
Sending initialized notification...
Handshake completed successfully


In [5]:
our_mcp_client.get_tools()
our_mcp_client.call_tool('get_weather', {'city': 'Berlin'})

Retrieving available tools...
Available tools: ['get_weather', 'set_weather']
Calling tool 'get_weather' with arguments: {'city': 'Berlin'}


{'content': [{'type': 'text', 'text': '20.0'}],
 'structuredContent': {'result': 20.0},
 'isError': False}

In [6]:
our_mcp_client.call_tool('set_weather', {'city': 'London', 'temp': 5.5})

Calling tool 'set_weather' with arguments: {'city': 'London', 'temp': 5.5}


{'content': [{'type': 'text', 'text': 'OK'}], 'isError': False}

In [7]:
our_mcp_client.call_tool('get_weather', {'city': 'London'})

Calling tool 'get_weather' with arguments: {'city': 'London'}


{'content': [{'type': 'text', 'text': '5.5'}],
 'structuredContent': {'result': 5.5},
 'isError': False}